# Scraping YouTube with Comments (one video)
Made by Ali Nurdin

In [32]:
# This section will import Selenium and Time
from selenium import webdriver
import time

In [33]:
# This section will open Web Driver
Driver = webdriver.Chrome()

In [34]:
# This section will get you to video based on given url
Driver.get("ENTER HERE") # Enter your video's url here
time.sleep(5)

First, you must skip the ads. This won't work if video's ads are not skipped. If you want to add auto skipped ads, please check another scraping script in this project (there, you will find the script for skipping the ads).

In [35]:
Column = Driver.find_element_by_tag_name("body").find_element_by_id("columns")
Primary_Inner = Column.find_element_by_id("primary").find_element_by_id("primary-inner")
Movie_Player = Primary_Inner.find_element_by_id("player").find_element_by_id("ytd-player").find_element_by_id("container").find_element_by_id("movie_player")

In [36]:
# This section will pause the video (it prevents auto-playing on Youtube).
Play_button = Movie_Player.find_element_by_class_name("ytp-chrome-controls").find_element_by_class_name("ytp-left-controls").find_element_by_css_selector("button.ytp-play-button.ytp-button")
Play_button.click()

In [37]:
# This section will assign video's information (such as duration, title, number of views, date, number of likes and dislike, etc.) to the following variable(s)/object(s).
Video_duration = Movie_Player.find_element_by_class_name("ytp-chrome-controls").find_element_by_class_name("ytp-left-controls").find_element_by_css_selector("div.ytp-time-display.notranslate").find_element_by_class_name("ytp-time-duration").text
Container = Primary_Inner.find_element_by_id("info").find_element_by_id("info-contents").find_element_by_id("container")
Title = Container.find_element_by_tag_name("h1").text
Info = Container.find_element_by_id("info")
Info_text = Info.find_element_by_id("info-text")
Number_of_views = Info_text.find_element_by_id("count").find_element_by_css_selector("span.view-count.style-scope.yt-view-count-renderer").text.split(" views")[0]
Date = Info_text.find_element_by_id("date").find_element_by_tag_name("yt-formatted-string").text
like_dislike_number = Info.find_element_by_id("menu-container").find_element_by_id("sentiment").find_element_by_tag_name("paper-tooltip").find_element_by_id("tooltip")
Driver.execute_script("arguments[0].setAttribute('class','style-scope paper-tooltip')", like_dislike_number)
number = Info.find_element_by_id("menu-container").find_element_by_id("sentiment").find_element_by_tag_name("paper-tooltip").find_element_by_id("tooltip").text
Driver.execute_script("arguments[0].setAttribute('class','style-scope paper-tooltip hidden')", like_dislike_number)
number = number.split(" / ")
Meta_container = Primary_Inner.find_element_by_id("meta").find_element_by_id("meta-contents").find_element_by_id("container")
Description = ""
try:
    Youtube_expander = Meta_container.find_element_by_tag_name("ytd-expander")
    Youtube_expander.find_element_by_id("more").click()
    Description = Youtube_expander.find_element_by_id("content").find_element_by_id("description").find_element_by_tag_name("yt-formatted-string").text
    Youtube_expander.find_element_by_id("less").click()
except:
    Description = Youtube_expander.find_element_by_id("content").find_element_by_id("description").find_element_by_tag_name("yt-formatted-string").text
    
Channel = Meta_container.find_element_by_id("top-row").find_element_by_tag_name("ytd-video-owner-renderer").find_element_by_id("upload-info")
Channel_name = Channel.find_element_by_id("channel-name").find_element_by_id("text-container").find_element_by_tag_name("a").text
Number_of_subscriber = Channel.find_element_by_id("owner-sub-count").text

In [38]:
# Just like its name, this function will handle the continuation of the comment section
def continuantions(Comment_section):
    try:
        Driver.execute_script("return arguments[0].scrollIntoView();", Comment_section.find_element_by_id("continuations").find_element_by_tag_name("yt-next-continuation"))
        time.sleep(3)
        return True
    except:
        return False
       
# Just like its name, this function will expand the reply section on the comment
def expand_replies(Replies_section):
    try:
        Replies_section.find_element_by_id("expander-contents").find_element_by_id("continuation").find_element_by_tag_name("yt-next-continuation").click()
        time.sleep(10)
        return True
    except:
        return False

In [39]:
# This section will initialize an array for accomodating comments on the given video.
Comment_data = []

In [40]:
# This section will scrap all comments and the replies on the given video.
Driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
time.sleep(5)
Comment_tag = Primary_Inner.find_element_by_id("comments")
Comment_section = Comment_tag.find_element_by_id("sections")
# comments_number = Comment_section.find_element_by_id("header").find_element_by_id("title").find_element_by_id("count").text.split(" Komentar")[0]

while continuantions(Comment_section):
    continue
    

Comment_contents = Comment_section.find_element_by_id("contents")
Comments = Comment_contents.find_elements_by_tag_name("ytd-comment-thread-renderer")

for i in range(0, len(Comments)):
    Comment = Comments[i]
    Comment_main_body = Comment.find_element_by_id("comment").find_element_by_id("body").find_element_by_id("main")
    author_name = Comment_main_body.find_element_by_id("header").find_element_by_id("header-author").find_element_by_id("author-text").text
    
    try:
        Comment_main_body.find_element_by_id("expander").find_element_by_id("more").click()
    except:
        pass
    
    Comment_content = Comment_main_body.find_element_by_id("expander").find_element_by_id("content").find_element_by_id("content-text").text
    date = Comment_main_body.find_element_by_id("header").find_element_by_id("header-author").find_element_by_tag_name("yt-formatted-string").text
    likes = Comment_main_body.find_element_by_id("action-buttons").find_element_by_id("toolbar").find_element_by_id("vote-count-left").text
    Comment_data.append({
        "author_name" : author_name,
        "date" : date,
        "comment" : Comment_content,
        "number_of_like" : likes
    })
    Replis_section = None
    try:
        Replies_section = Comment.find_element_by_tag_name("ytd-comment-replies-renderer").find_element_by_id("expander")
        Replies_section.find_element_by_id("more-replies").click() # buat klik "lihat balasan"
        time.sleep(5)
    except:
        continue
    while expand_replies(Replies_section):
        continue

    try:
        Replies = Replies_section.find_element_by_id("expander-contents").find_element_by_id("loaded-replies").find_elements_by_tag_name("ytd-comment-renderer")
    except:
        continue

    for reply in Replies:
        reply_main_body = reply.find_element_by_id("body").find_element_by_id("main")
        reply_author_name = reply_main_body.find_element_by_id("header").find_element_by_id("header-author").find_element_by_id("author-text").text
        reply_Comment_content = reply_main_body.find_element_by_id("expander").find_element_by_id("content").find_element_by_id("content-text").text
        reply_date = reply_main_body.find_element_by_id("header").find_element_by_id("header-author").find_element_by_tag_name("yt-formatted-string").text
        reply_likes = reply_main_body.find_element_by_id("action-buttons").find_element_by_id("toolbar").find_element_by_id("vote-count-left").text
        Comment_data.append({
            "author-name" : reply_author_name,
            "date" : reply_date,
            "comment" : reply_Comment_content,
            "number_of_like" : reply_likes
        })
    try:
        Replies_section.find_element_by_id("less-replies").click()
    except:
        continue
    

In [41]:
# This section will assign video id to the following variable(s)/object(s).
Video_id = Driver.current_url.split("?v=")[-1]

In [42]:
# This section will initialize the array for accomodating data from youtube
Youtube_video = []

In [43]:
# This section will combine all data which have obtained together.
Youtube_video.append({
    "title_id" : Video_id,
    "title" : Title,
    "duration" : Video_duration,
    "date" : Date,
    "channel" : Channel_name,
    "description" : Description,
    "number_of_subscriber" : "1,210,000",
    "number_of_viewer" : Number_of_views,
    "number_of_like" : number[0],
    "number_of_dislike" : number[1],
    "number_of_comment" : len(Comment_data),
    "comments" : Comment_data
})

In [44]:
# Importing json
import json

In [45]:
# This section will enter the data that has been obtained into the json file.
file = open("JSON FILE NAME.json", 'w')
json.dump(Youtube_video, file)
file.close()